# Dfsu 2D

2D horizontal. 

See [Dfsu in MIKE IO Documentation](https://dhi.github.io/mikeio/dfsu-mesh-overview.html)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mikeio

## Basic properties

:::{note}
Example data can be found in the `mini_book/data` folder in this [zip-file](https://github.com/DHI/book-learn-mikeio-fmskill/archive/refs/heads/main.zip).
:::

In [ ]:
fn = "data/oresundHD_run1.dfsu"
dfs = mikeio.open(fn)
dfs

In [ ]:
dfs.geometry

In [ ]:
dfs.geometry.plot();

In [ ]:
dfs.n_items

In [ ]:
dfs.projection_string

In [ ]:
dfs.timestep    # in seconds ...

In [ ]:
ds = dfs.read(items="Surface elevation")
ds.shape

In [ ]:
da = ds["Surface elevation"]   # =ds[0] = ds.Surface_elevation
da

In [ ]:
da.plot();

### Subset data

In [ ]:
ze = dfs.element_coordinates[:,2]    # third column is depth
ze<-20

In [ ]:
elems = np.where(ze<-20)[0]
ds_sub = dfs.read(items=0, time=-1, elements=elems)    # read only data where depth is more than 20m
ds_sub

In [ ]:
ax = ds_sub[0].plot()
dfs.geometry.plot.mesh(ax=ax);

Alternatively, we can use boolean masking directly on the DataArray, like this (see also [DataArray_Masking.ipynb](https://github.com/DHI/mikeio/blob/main/notebooks/DataArray%20-%20Masking.ipynb)):

In [ ]:
dac = ds[0].isel(time=-1).copy()
dac[ze>=-20] = np.nan
dac.plot(show_mesh=True);

## Create new dfsu from mesh

In [ ]:
fn = "data/two_elements.mesh"
dfs = mikeio.open(fn)

In [ ]:
data = 36.5 * np.ones(dfs.n_elements)            # set all elements to 36.5
time = pd.date_range("2020-1-1", periods=1)      # arbitrary date 
item = mikeio.ItemInfo(mikeio.EUMType.Chezy_No)
da = mikeio.DataArray(data, time=time, item=item, geometry=dfs.geometry)

In [ ]:
da.to_dfs("chezy_map.dfsu")

In [ ]:
dfs = mikeio.open("chezy_map.dfsu")
dfs.read()[0][0,:]

## Visualisation

In [ ]:
dfs = mikeio.open("data/oresundHD_run1.dfsu")
dfs

The Dfsu geometry plot the same as the mesh. (plot the elements and color them according to the bathymetry).

The DataArray can be used to plot other data, such as surface elevation.

In [ ]:
dfs.geometry.plot();

In [ ]:
ds = dfs.read()
ds

In [ ]:
wl_laststep = ds["Surface elevation"].isel(time=-1)   # DataArray
wl_laststep


In order to customize the plot we can take return the axis and add additional things, like markers and a legend.

In [ ]:
ax = wl_laststep.plot(cmap="winter", show_mesh=True, figsize=(12,12))
ax.scatter(x=350000, y=6.15e6, marker='*', s=200, label="Location of bouy")
ax.legend();

In order to create subplots, we can supply the axis as an argument to plot.

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize = (12,7))

da = ds["Surface elevation"].isel(time=-1)
da.plot.contourf(ax=ax1, title="ax=ax1")

da = ds["V velocity"].isel(time=-1)
da.plot.contourf(ax=ax2, title="ax=ax2", label="Northward velocity [m/s]");

## More examples

See the [MIKE IO example notebooks](https://dhi.github.io/mikeio-examples/search.html) which has "dfsu" in the name.

